### Compile Edge Lists from Medicare and commercial claims: 
##### Monthly claims by physician & facility from all available data sources. 
##### _____________________________________________________________________________________

Import relevant libraries

In [1]:
import pandas as pd 
import numpy as np 

from google.cloud import bigquery 
import pyodbc

from datetime import datetime 

Establish connections to BigQuery and SSMS

In [2]:
bq_conn = bigquery.Client() 
ssms_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=Database5;DATABASE=DefinitiveHC;Trusted_Connection=yes;')
p02_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=DHCSQLP02;DATABASE=DS_WORK;Trusted_Connection=yes;')

#### COMMERCIAL CLAIMS

##### Rendering-Billing

In [3]:
comm_ren_bil_sql = """
CREATE  OR REPLACE TABLE DS_WORK.RENDERING_BILLING_AFFILIATIONS
AS
SELECT  RENDERING_NPI
    ,   CLAIM_MONTH
    ,   A.BILLING_NPI
    ,   A.CLAIMS 
    ,   A.MIN_SERVICE_TO_DATE
    ,   A.MAX_SERVICE_TO_DATE 
FROM    (
        SELECT  RENDERING_NPI
            ,   CLAIM_YEAR 
            ,   SAFE_CAST(CONCAT(CAST(CLAIM_YEAR AS STRING), '-', CAST(EXTRACT(MONTH FROM SERVICE_TO_DATE) AS STRING), '-', '01') AS DATE) AS CLAIM_MONTH
            ,   BILLING_NPI
            ,   COUNT(*) AS CLAIMS
            ,   MIN(SERVICE_TO_DATE) AS MIN_SERVICE_TO_DATE
            ,   MAX(SERVICE_TO_DATE) AS MAX_SERVICE_TO_DATE
        FROM   (
                SELECT DISTINCT CLAIM_YEAR
                    ,  SAFE_CAST(CONCAT(SUBSTR(SAFE_CAST(SERVICE_TO_DATE AS STRING), 1, 4), '-', SUBSTR(SAFE_CAST(SERVICE_TO_DATE AS STRING), 5, 2), '-', SUBSTR(SAFE_CAST(SERVICE_TO_DATE AS STRING), 7, 2)) AS DATE) AS SERVICE_TO_DATE
                    ,  B.NPI AS RENDERING_NPI
                    ,  C.NPI AS BILLING_NPI
                    ,  A.DPID 
                FROM   CommercialHealthClaims.CLMS_EntityData A
                JOIN   CommercialHealthClaims.LOOKUP_DEFHC_ID B
                ON     A.RENDERING_NPIID = B.NPIID 
                JOIN   CommercialHealthClaims.LOOKUP_DEFHC_ID C
                ON     A.BILLING_NPIID = C.NPIID
                WHERE  RENDERING_NPIID IS NOT NULL 
                AND    BILLING_NPIID IS NOT NULL 
                AND    RENDERING_NPIID <> BILLING_NPIID 
                AND    CLAIM_YEAR >= 2016
                ) 
        GROUP   BY RENDERING_NPI
            ,   CLAIM_YEAR 
            ,   SAFE_CAST(CONCAT(CAST(CLAIM_YEAR AS STRING), '-', CAST(EXTRACT(MONTH FROM SERVICE_TO_DATE) AS STRING), '-', '01') AS DATE)
            ,   BILLING_NPI
        ) A 
"""

bq_conn.query(comm_ren_bil_sql).result()

##### Rendering-Facility

In [4]:
comm_ren_fac_sql = """
CREATE  OR REPLACE TABLE DS_WORK.RENDERING_SERVICE_FACILITY_AFFILIATIONS
AS
SELECT  RENDERING_NPI
    ,   CLAIM_MONTH
    ,   A.SERVICE_FACILITY_NPI
    ,   A.CLAIMS 
    ,   A.MIN_SERVICE_TO_DATE
    ,   A.MAX_SERVICE_TO_DATE 
FROM    (
        SELECT  RENDERING_NPI
            ,   CLAIM_YEAR 
            ,   SAFE_CAST(CONCAT(CAST(CLAIM_YEAR AS STRING), '-', CAST(EXTRACT(MONTH FROM SERVICE_TO_DATE) AS STRING), '-', '01') AS DATE) AS CLAIM_MONTH
            ,   SERVICE_FACILITY_NPI
            ,   COUNT(*) AS CLAIMS
            ,   MIN(SERVICE_TO_DATE) AS MIN_SERVICE_TO_DATE
            ,   MAX(SERVICE_TO_DATE) AS MAX_SERVICE_TO_DATE
        FROM   (
                SELECT DISTINCT CLAIM_YEAR
                    ,  SAFE_CAST(CONCAT(SUBSTR(SAFE_CAST(SERVICE_TO_DATE AS STRING), 1, 4), '-', SUBSTR(SAFE_CAST(SERVICE_TO_DATE AS STRING), 5, 2), '-', SUBSTR(SAFE_CAST(SERVICE_TO_DATE AS STRING), 7, 2)) AS DATE) AS SERVICE_TO_DATE
                    ,  B.NPI AS RENDERING_NPI
                    ,  C.NPI AS SERVICE_FACILITY_NPI
                    ,  A.DPID
                FROM   CommercialHealthClaims.CLMS_EntityData A 
                JOIN   CommercialHealthClaims.LOOKUP_DEFHC_ID B
                ON     A.RENDERING_NPIID = B.NPIID 
                JOIN   CommercialHealthClaims.LOOKUP_DEFHC_ID C
                ON     A.SERVICE_FACILITY_NPIID = C.NPIID
                WHERE  RENDERING_NPIID IS NOT NULL 
                AND    SERVICE_FACILITY_NPIID IS NOT NULL 
                AND    SERVICE_FACILITY_NPIID <> IFNULL(BILLING_NPIID, 0) 
                AND    CLAIM_YEAR >= 2016
                ) 
        GROUP   BY RENDERING_NPI
            ,   CLAIM_YEAR 
            ,   SAFE_CAST(CONCAT(CAST(CLAIM_YEAR AS STRING), '-', CAST(EXTRACT(MONTH FROM SERVICE_TO_DATE) AS STRING), '-', '01') AS DATE)
            ,   SERVICE_FACILITY_NPI
        ) A 
""" 

bq_conn.query(comm_ren_fac_sql).result()

#### MEDICARE CLAIMS

Institutional (Inpatient, Outpatient, SNF, HHA, Hospice)

In [5]:
medicare_years_df = pd.DataFrame()
medicare_years_qtrly_df = pd.DataFrame()

for db in(list(pd.read_sql("""SELECT [name] AS DB FROM sys.databases WHERE [name] LIKE 'MEDICARE_SAF_20%' """, p02_conn)['DB'].values)):
    medicare_years_df = medicare_years_df.append(pd.read_sql("""SELECT {yr} AS CLAIM_YEAR FROM {db}.INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME ='CLAIMS_FILE' AND {yr} > 2015 """.format(db=db, yr=db[-4:]), p02_conn))
    
for db in(list(pd.read_sql("""SELECT [name] AS DB FROM sys.databases WHERE [name] LIKE 'MEDICARE_SAF_20%' """, p02_conn)['DB'].values)):
    medicare_years_qtrly_df = medicare_years_qtrly_df.append(pd.read_sql("""SELECT {yr} AS CLAIM_YEAR FROM {db}.INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME ='CLAIMS_FILE_QTRLY' AND {yr} > 2015 """.format(db=db, yr=db[-4:]), p02_conn))
    

In [6]:
medicare_years = list(medicare_years_df['CLAIM_YEAR'].values)
medicare_years_qtrly = set(medicare_years_qtrly_df['CLAIM_YEAR'].values) - set(medicare_years)
medicare_years_qtrly = list(medicare_years_qtrly)

In [7]:
# print(medicare_years)
# print(medicare_years_qtrly)

In [8]:
#truncate tables in BQ 

In [9]:
bq_conn.query("""DELETE FROM DS_WORK.MCR_ATTENDING_ORGANIZATION_AFFILIATIONS WHERE 1=1""").result()  
bq_conn.query("""DELETE FROM DS_WORK.MCR_OPERATING_ORGANIZATION_AFFILIATIONS WHERE 1=1""").result()  
bq_conn.query("""DELETE FROM DS_WORK.MCR_RENDERING_ORGANIZATION_AFFILIATIONS WHERE 1=1""").result() 

##### Attending-Organization

In [10]:
# populate table in BQ 

In [11]:
for yr in medicare_years: 
    mcr_sql = """
    SELECT  AT_PHYSN_NPI
        ,   CLAIM_MONTH
        ,   A.ORG_NPI_NUM
        ,   A.CLAIMS 
        ,   A.MIN_SERVICE_TO_DATE
        ,   A.MAX_SERVICE_TO_DATE 
    FROM    (
            SELECT  AT_PHYSN_NPI
                ,   CLAIM_YEAR 
                ,   CAST(CAST(YEAR(SERVICE_TO_DATE) AS VARCHAR) + '-' + CAST(MONTH(SERVICE_TO_DATE) AS VARCHAR) + '-' + '01' AS DATE) AS CLAIM_MONTH 
                ,   ORG_NPI_NUM
                ,   COUNT(*) AS CLAIMS
                ,   MIN(SERVICE_TO_DATE) AS MIN_SERVICE_TO_DATE
                ,   MAX(SERVICE_TO_DATE) AS MAX_SERVICE_TO_DATE
            FROM   (
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(AT_PHYSN_NPI  AS INT) AS AT_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.inp_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(AT_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    AT_PHYSN_NPI <> ORG_NPI_NUM 
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(AT_PHYSN_NPI  AS INT) AS AT_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.out_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(AT_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    AT_PHYSN_NPI <> ORG_NPI_NUM 
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(AT_PHYSN_NPI  AS INT) AS AT_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.snf_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(AT_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    AT_PHYSN_NPI <> ORG_NPI_NUM 
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(AT_PHYSN_NPI  AS INT) AS AT_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.hha_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(AT_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    AT_PHYSN_NPI <> ORG_NPI_NUM 
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(AT_PHYSN_NPI  AS INT) AS AT_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.hosp_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(AT_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    AT_PHYSN_NPI <> ORG_NPI_NUM 
                    ) A
            GROUP   BY AT_PHYSN_NPI
                ,   CLAIM_YEAR 
                ,   CAST(CAST(YEAR(SERVICE_TO_DATE) AS VARCHAR) + '-' + CAST(MONTH(SERVICE_TO_DATE) AS VARCHAR) + '-' + '01' AS DATE) 
                ,   ORG_NPI_NUM
            ) A 
    WHERE   AT_PHYSN_NPI <> 0 
    AND     ORG_NPI_NUM <> 0 
    """.format(yr=yr)
    
    print(datetime.now())
    print('Attending-organization data for {yr}'.format(yr=yr))
    df = pd.read_sql(mcr_sql, p02_conn)
    print(datetime.now())
    
    # push to BQ
    dataframe = df
    tablename = 'stable-healer-231019.DS_WORK.MCR_ATTENDING_ORGANIZATION_AFFILIATIONS'

    job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
    job.result()
    

2020-07-01 12:12:01.390747
Attending-organization data for 2016
2020-07-01 12:24:39.594222


/usr/local/lib/python3.6/site-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'CLAIM_MONTH'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


2020-07-01 12:29:15.948250
Attending-organization data for 2017
2020-07-01 12:43:56.977308
2020-07-01 12:48:56.105934
Attending-organization data for 2018
2020-07-01 13:01:16.963571


In [12]:
for yr in medicare_years_qtrly: 
    
    mcr_sql = """
    SELECT  AT_PHYSN_NPI
        ,   CLAIM_MONTH
        ,   A.ORG_NPI_NUM
        ,   A.CLAIMS 
        ,   A.MIN_SERVICE_TO_DATE
        ,   A.MAX_SERVICE_TO_DATE 
    FROM    (
            SELECT  AT_PHYSN_NPI
                ,   CLAIM_YEAR 
                ,   CAST(CAST(YEAR(SERVICE_TO_DATE) AS VARCHAR) + '-' + CAST(MONTH(SERVICE_TO_DATE) AS VARCHAR) + '-' + '01' AS DATE) AS CLAIM_MONTH 
                ,   ORG_NPI_NUM
                ,   COUNT(*) AS CLAIMS
                ,   MIN(SERVICE_TO_DATE) AS MIN_SERVICE_TO_DATE
                ,   MAX(SERVICE_TO_DATE) AS MAX_SERVICE_TO_DATE
            FROM   (
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(AT_PHYSN_NPI  AS INT) AS AT_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.inp_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(AT_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    AT_PHYSN_NPI <> ORG_NPI_NUM 
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(AT_PHYSN_NPI  AS INT) AS AT_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.out_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(AT_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    AT_PHYSN_NPI <> ORG_NPI_NUM 
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(AT_PHYSN_NPI  AS INT) AS AT_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.snf_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(AT_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    AT_PHYSN_NPI <> ORG_NPI_NUM 
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(AT_PHYSN_NPI  AS INT) AS AT_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.hha_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(AT_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    AT_PHYSN_NPI <> ORG_NPI_NUM 
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(AT_PHYSN_NPI  AS INT) AS AT_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.hosp_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(AT_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    AT_PHYSN_NPI <> ORG_NPI_NUM 
                    ) A
            GROUP   BY AT_PHYSN_NPI
                ,   CLAIM_YEAR 
                ,   CAST(CAST(YEAR(SERVICE_TO_DATE) AS VARCHAR) + '-' + CAST(MONTH(SERVICE_TO_DATE) AS VARCHAR) + '-' + '01' AS DATE) 
                ,   ORG_NPI_NUM
            ) A 
    WHERE   AT_PHYSN_NPI <> 0 
    AND     ORG_NPI_NUM <> 0 
    """.format(yr=yr)
    
    
    print(datetime.now())
    print('Attending-organization data for {yr}'.format(yr=yr))
    df = pd.read_sql(mcr_sql, p02_conn)
    print(datetime.now())
    
    # push to BQ
    dataframe = df
    tablename = 'stable-healer-231019.DS_WORK.MCR_ATTENDING_ORGANIZATION_AFFILIATIONS'

    job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
    job.result()

2020-07-01 13:05:17.215799
Attending-organization data for 2019
2020-07-01 13:17:40.253035


##### Operating-Organization

In [13]:
# populate table in BQ 

In [14]:
for yr in medicare_years: 

    mcr_sql2 = """SELECT  OP_PHYSN_NPI
        ,   CLAIM_MONTH
        ,   A.ORG_NPI_NUM
        ,   A.CLAIMS 
        ,   A.MIN_SERVICE_TO_DATE
        ,   A.MAX_SERVICE_TO_DATE 
    FROM    (
            SELECT  OP_PHYSN_NPI
                ,   CLAIM_YEAR 
                ,   CAST(CAST(YEAR(SERVICE_TO_DATE) AS VARCHAR) + '-' + CAST(MONTH(SERVICE_TO_DATE) AS VARCHAR) + '-' + '01' AS DATE) AS CLAIM_MONTH 
                ,   ORG_NPI_NUM
                ,   COUNT(*) AS CLAIMS
                ,   MIN(SERVICE_TO_DATE) AS MIN_SERVICE_TO_DATE
                ,   MAX(SERVICE_TO_DATE) AS MAX_SERVICE_TO_DATE
            FROM   (
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(OP_PHYSN_NPI  AS INT) AS OP_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.inp_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(OP_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    OP_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    OP_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(OP_PHYSN_NPI  AS INT) AS OP_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.out_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(OP_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    OP_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    OP_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(OP_PHYSN_NPI  AS INT) AS OP_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.snf_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(OP_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    OP_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    OP_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(OP_PHYSN_NPI  AS INT) AS OP_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.hha_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(OP_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    OP_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    OP_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(OP_PHYSN_NPI  AS INT) AS OP_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.hosp_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(OP_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    OP_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    OP_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    ) A
            GROUP   BY OP_PHYSN_NPI
                ,   CLAIM_YEAR 
                ,   CAST(CAST(YEAR(SERVICE_TO_DATE) AS VARCHAR) + '-' + CAST(MONTH(SERVICE_TO_DATE) AS VARCHAR) + '-' + '01' AS DATE) 
                ,   ORG_NPI_NUM
            ) A """.format(yr=yr)
    
    print(datetime.now())
    print('Operating-org data for {yr}'.format(yr=yr))
    df = pd.read_sql(mcr_sql2, p02_conn)
    print(datetime.now())
    
    #push to BQ
    dataframe = df
    tablename = 'stable-healer-231019.DS_WORK.MCR_OPERATING_ORGANIZATION_AFFILIATIONS'

    job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
    job.result()

2020-07-01 13:21:55.034323
Operating-org data for 2016
2020-07-01 13:31:36.149411
2020-07-01 13:32:47.743530
Operating-org data for 2017
2020-07-01 13:38:25.284389
2020-07-01 13:39:04.015557
Operating-org data for 2018
2020-07-01 13:41:02.426834


In [16]:
for yr in medicare_years_qtrly: 
    
    mcr_sql2 = """SELECT  OP_PHYSN_NPI
        ,   CLAIM_MONTH
        ,   A.ORG_NPI_NUM
        ,   A.CLAIMS 
        ,   A.MIN_SERVICE_TO_DATE
        ,   A.MAX_SERVICE_TO_DATE 
    FROM    (
            SELECT  OP_PHYSN_NPI
                ,   CLAIM_YEAR 
                ,   CAST(CAST(YEAR(SERVICE_TO_DATE) AS VARCHAR) + '-' + CAST(MONTH(SERVICE_TO_DATE) AS VARCHAR) + '-' + '01' AS DATE) AS CLAIM_MONTH 
                ,   ORG_NPI_NUM
                ,   COUNT(*) AS CLAIMS
                ,   MIN(SERVICE_TO_DATE) AS MIN_SERVICE_TO_DATE
                ,   MAX(SERVICE_TO_DATE) AS MAX_SERVICE_TO_DATE
            FROM   (
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(OP_PHYSN_NPI  AS INT) AS OP_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.inp_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(OP_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    OP_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    OP_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(OP_PHYSN_NPI  AS INT) AS OP_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.out_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(OP_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    OP_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    OP_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(OP_PHYSN_NPI  AS INT) AS OP_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.snf_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(OP_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    OP_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    OP_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(OP_PHYSN_NPI  AS INT) AS OP_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.hha_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(OP_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    OP_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    OP_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(OP_PHYSN_NPI  AS INT) AS OP_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.hosp_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(OP_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    OP_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    OP_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    ) A
            GROUP   BY OP_PHYSN_NPI
                ,   CLAIM_YEAR 
                ,   CAST(CAST(YEAR(SERVICE_TO_DATE) AS VARCHAR) + '-' + CAST(MONTH(SERVICE_TO_DATE) AS VARCHAR) + '-' + '01' AS DATE) 
                ,   ORG_NPI_NUM
            ) A """.format(yr=yr)
    
    print(datetime.now())
    print('Operating-org data for {yr}'.format(yr=yr))
    df = pd.read_sql(mcr_sql2, p02_conn)
    print(datetime.now())
    
    #push to BQ
    dataframe = df
    tablename = 'stable-healer-231019.DS_WORK.MCR_OPERATING_ORGANIZATION_AFFILIATIONS'

    job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
    job.result()

2020-07-01 13:44:59.555105
Operating-org data for 2019
2020-07-01 13:47:26.322270


##### Rendering-Organization

In [17]:
# populate table in BQ

In [19]:
for yr in medicare_years: 
    
    mcr_sql3 = """SELECT  RNDRNG_PHYSN_NPI AS RENDERING_NPI
        ,   CLAIM_MONTH
        ,   A.ORG_NPI_NUM
        ,   A.CLAIMS 
        ,   A.MIN_SERVICE_TO_DATE
        ,   A.MAX_SERVICE_TO_DATE 
    FROM    (
            SELECT  RNDRNG_PHYSN_NPI
                ,   CLAIM_YEAR 
                ,   CAST(CAST(YEAR(SERVICE_TO_DATE) AS VARCHAR) + '-' + CAST(MONTH(SERVICE_TO_DATE) AS VARCHAR) + '-' + '01' AS DATE) AS CLAIM_MONTH 
                ,   ORG_NPI_NUM
                ,   COUNT(*) AS CLAIMS
                ,   MIN(SERVICE_TO_DATE) AS MIN_SERVICE_TO_DATE
                ,   MAX(SERVICE_TO_DATE) AS MAX_SERVICE_TO_DATE
            FROM   (
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) AS RNDRNG_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.inp_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    RNDRNG_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(OP_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) AS RNDRNG_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.out_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    RNDRNG_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(OP_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) AS RNDRNG_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.snf_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    RNDRNG_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(OP_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) AS RNDRNG_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.hha_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    RNDRNG_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(OP_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) AS RNDRNG_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.hosp_claimsk_lds_100_{yr}
                    WHERE  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    RNDRNG_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(OP_PHYSN_NPI,0)
                    ) A
            GROUP   BY RNDRNG_PHYSN_NPI
                ,   CLAIM_YEAR 
                ,   CAST(CAST(YEAR(SERVICE_TO_DATE) AS VARCHAR) + '-' + CAST(MONTH(SERVICE_TO_DATE) AS VARCHAR) + '-' + '01' AS DATE) 
                ,   ORG_NPI_NUM
            ) A  """.format(yr=yr)
    
    print(datetime.now())
    print('Rendering-org data for {yr}'.format(yr=yr))
    df = pd.read_sql(mcr_sql3, p02_conn)
    print(datetime.now())
    
    #push to BQ
    dataframe = df
    tablename = 'stable-healer-231019.DS_WORK.MCR_RENDERING_ORGANIZATION_AFFILIATIONS'

    job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
    job.result()

2020-07-01 13:48:38.510771
Rendering-org data for 2016
2020-07-01 13:52:21.753501
2020-07-01 13:52:35.295077
Rendering-org data for 2017
2020-07-01 13:57:33.846294
2020-07-01 13:57:53.736680
Rendering-org data for 2018
2020-07-01 13:59:13.915718


In [20]:
for yr in medicare_years_qtrly: 
    
    mcr_sql3 = """SELECT  RNDRNG_PHYSN_NPI AS RENDERING_NPI
        ,   CLAIM_MONTH
        ,   A.ORG_NPI_NUM
        ,   A.CLAIMS 
        ,   A.MIN_SERVICE_TO_DATE
        ,   A.MAX_SERVICE_TO_DATE 
    FROM    (
            SELECT  RNDRNG_PHYSN_NPI
                ,   CLAIM_YEAR 
                ,   CAST(CAST(YEAR(SERVICE_TO_DATE) AS VARCHAR) + '-' + CAST(MONTH(SERVICE_TO_DATE) AS VARCHAR) + '-' + '01' AS DATE) AS CLAIM_MONTH 
                ,   ORG_NPI_NUM
                ,   COUNT(*) AS CLAIMS
                ,   MIN(SERVICE_TO_DATE) AS MIN_SERVICE_TO_DATE
                ,   MAX(SERVICE_TO_DATE) AS MAX_SERVICE_TO_DATE
            FROM   (
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) AS RNDRNG_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.inp_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    RNDRNG_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(OP_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) AS RNDRNG_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.out_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    RNDRNG_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(OP_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) AS RNDRNG_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.snf_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    RNDRNG_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(OP_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) AS RNDRNG_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.hha_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    RNDRNG_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(OP_PHYSN_NPI,0)
                    UNION ALL 
                    SELECT DISTINCT {yr} AS CLAIM_YEAR
                        ,  CAST(CLM_THRU_DT AS DATE) AS SERVICE_TO_DATE
                        ,  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) AS RNDRNG_PHYSN_NPI
                        ,  TRY_CAST(ORG_NPI_NUM AS INT) AS ORG_NPI_NUM
                        ,  DESY_SORT_KEY
                    FROM   MEDICARE_SAF_{yr}.dbo.hosp_claimsk_lds_100_{yr}_q
                    WHERE  TRY_CAST(RNDRNG_PHYSN_NPI  AS INT) IS NOT NULL 
                    AND    TRY_CAST(ORG_NPI_NUM AS INT) IS NOT NULL 
                    AND    RNDRNG_PHYSN_NPI <> ORG_NPI_NUM 
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(AT_PHYSN_NPI,0)
                    AND    RNDRNG_PHYSN_NPI <> ISNULL(OP_PHYSN_NPI,0)
                    ) A
            GROUP   BY RNDRNG_PHYSN_NPI
                ,   CLAIM_YEAR 
                ,   CAST(CAST(YEAR(SERVICE_TO_DATE) AS VARCHAR) + '-' + CAST(MONTH(SERVICE_TO_DATE) AS VARCHAR) + '-' + '01' AS DATE) 
                ,   ORG_NPI_NUM
            ) A  """.format(yr=yr)
    
    print(datetime.now())
    print('Rendering-org data for {yr}'.format(yr=yr))
    df = pd.read_sql(mcr_sql3, p02_conn)
    print(datetime.now())
    
    #push to BQ
    dataframe = df
    tablename = 'stable-healer-231019.DS_WORK.MCR_RENDERING_ORGANIZATION_AFFILIATIONS'

    job = bq_conn.load_table_from_dataframe(dataframe, destination = tablename)
    job.result()

2020-07-01 13:59:26.315011
Rendering-org data for 2019
2020-07-01 14:01:33.633502
